# Process OGC Web Feature Service

This notebook uses an endpoint URL, passed either as an 'endpoint' parameter when the notebook is called, or assigned in cell 5.



#for some reason the env variable is not getting set when the book is first opened, have to set manually
#WFS driver doesn't work
#!set GDAL_DATA=E:\EPrograms\Anaconda\envs\py36\Library\share\gdal
!set GDAL_DATA


%%javascript
//check if there is an endpoint parameter in the notebook call

function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("endpoint='".concat(getQueryStringValue("endpoint")).concat("'"));
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

In [19]:
import sys
import osgeo.gdal as gdal
import osgeo.ogr as ogr
#try:
#    from osgeo import ogr, gdal
#except:
#    sys.exit('ERROR: cannot find GDAL/OGR modules')
    
from ipywidgets import interact

import requests
import json
from pandas import json_normalize
maxfeatures=50000
endpoint = None
full_notebook_url = None

#import numpy as np
#import pandas as pd
#import xmltodict
#from ipywidgets import interactive, fixed, interact_manual
#import ipywidgets as widgets

In [2]:
def the_envelope(geom):
    (minX, maxX, minY, maxY) = geom.GetEnvelope()
    # calculate a bounding box geometry for the given geometry argument
    # Create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(minX, minY)
    ring.AddPoint(maxX, minY)
    ring.AddPoint(maxX, maxY)
    ring.AddPoint(minX, maxY)
    ring.AddPoint(minX, minY)

    # Create polygon
    poly_envelope = ogr.Geometry(ogr.wkbPolygon)
    poly_envelope.AddGeometry(ring)
    return poly_envelope

In [3]:
def testurl(theurl):
    #try HEAD first in case the response document is big
    r = requests.head(theurl)
    if (r.status_code != requests.codes.ok):
        #check GET in case is an incomplete http implementation
        r = requests.get(theurl)
        if (r.status_code == requests.codes.ok):
            return True
        else:
            return False
    else:
        return True

In [4]:
#url_partitioned = full_notebook_url.partition('ISOmetadata-ExtractDistributions.ipynb')
#base_url = url_partitioned[0];

if(endpoint is not None and len(endpoint)>0):
    print('Got endpoint parameter:',endpoint)
else: 
    # some USGS ScienceBase Geoserver WFS endpoints:
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c54be4b0aa151574a8dc'
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/5342c5fce4b0aa151574a8ed'
    #endpoint='https://www.sciencebase.gov/catalogMaps/mapping/ows/4f4e49cfe4b07f02db5da90e'

    # some NGDS end points
    endpoint='https://geoserver.hydroshare.org/geoserver/HS-c8427d7a30f0440b9773d17b7369fb69/wfs'
    #endpoint='http://services.azgs.az.gov/arcgis/services/aasggeothermal/AZActiveFaults/MapServer/WFSServer'

    #EarthChem
    #endpoint='http://ecp.iedadata.org/wfs/ows?service=wfs'

    #smu thermal conductivity, geoserver
    #endpoint='http://geothermal.smu.edu:9000/geoserver/aasg-thermalconductivity/ows'

    #NOAA Watch/warning polygons
    #endpoint = 'https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Forecasts_Guidance_Warnings/watch_warn_adv/MapServer/WFSServer'
    print('Assign endpoint:', endpoint)
# limit the number of features that will be processed


Assign endpoint: https://geoserver.hydroshare.org/geoserver/HS-c8427d7a30f0440b9773d17b7369fb69/wfs


In [28]:
# see https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html 
# for documentation on ogr usage
## org driver available?
driverName = "WFS"
wfs_drv = ogr.GetDriverByName(driverName)

if wfs_drv is None:
    print ("%s driver not available.\n" % driverName)
else:
    print  ("%s driver IS available.\n" % wfs_drv.name)

WFS driver IS available.



In [6]:
# !set GDAL_DATA=E:\EPrograms\Anaconda\Library\share\gdal
# !set GDAL_DATA


In [7]:
# Speeds up querying WFS capabilities for services with alot of layers
gdal.SetConfigOption('OGR_WFS_LOAD_MULTIPLE_LAYER_DEFN', 'NO')

# Set config for paging. Works on WFS 2.0 services and WFS 1.0 and 1.1 with some other services.
gdal.SetConfigOption('OGR_WFS_PAGING_ALLOWED', 'YES')
gdal.SetConfigOption('OGR_WFS_PAGE_SIZE', '10000')


#test that service is up with getcapabilities request

capurl=endpoint + '?service=wfs&request=getCapabilities'

if testurl(capurl):
    # Open the webservice--check if responds to getCapabilities
    wfs_ds = wfs_drv.Open('WFS:' + endpoint)
    #wfs_ds = wfs_drv.Open("WFS:http://services.azgs.az.gov/arcgis/services/aasggeothermal/CAaqWellChemistry/MapServer/WFSServer")
    #wfs_ds = wfs_drv.Open(endpoint + '?service=wfs&request=getCapabilities&version=1.1.0')
    if wfs_ds is None:
        print('ERROR: ogr driver cannot open WFS datasource')
else:
    print('ERROR: service not responding')


In [8]:
capurl
# ogrinfo -ro WFS:http://www2.dmsolutions.ca/cgi-bin/mswfs_gmap


'https://geoserver.hydroshare.org/geoserver/HS-c8427d7a30f0440b9773d17b7369fb69/wfs?service=wfs&request=getCapabilities'

In [9]:
# iterate over available layers
for i in range(wfs_ds.GetLayerCount()):
    layer = wfs_ds.GetLayerByIndex(i)
    srs = layer.GetSpatialRef()
    print ('Layer: %s, Features: %s, SR: %s...' % (layer.GetName(), layer.GetFeatureCount(), srs.ExportToWkt()[0:50]))

    # iterate over features
    #fcount = 1
    #feat = layer.GetNextFeature()
    #while feat is not None:
    #    feat = layer.GetNextFeature()
    #    fcount = fcount + 1
        # do something more..
    #feat = None
    print ('feature count: ',layer.GetFeatureCount())

Layer: HS-c8427d7a30f0440b9773d17b7369fb69:watersheds, Features: 33, SR: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84"...
feature count:  33


#gdal call to get the capabilities document
caps=wfs_ds.GetLayerByName('WFSGetCapabilities')
#print the number of feature types reported in the capabilities document
print(caps.GetFeatureCount())

for feat in caps:
    jsonwrap= json.loads(feat.ExportToJson())
    
capsxml=jsonwrap['properties']['content']
capsjson=xmltodict.parse(capsxml)
capsjson=json.dumps(capsjson, indent=2)
#print(capsjson)

Want to let user pick the layer to work with.

In [11]:

feat_list={}
for i in range(wfs_ds.GetLayerCount()):
    layer = wfs_ds.GetLayerByIndex(i)
    feat_list[layer.GetName()] = layer.GetName()


def f(feat_menu):
    return feat_menu
# out = interact(f, feat_menu=nb_menu);
out = interact(f, feat_menu=feat_list.keys());

#print("interact out: ", out)

interactive(children=(Dropdown(description='feat_menu', options=('HS-c8427d7a30f0440b9773d17b7369fb69:watershe…

In [12]:
# Get a specific layer

chosenlayer= feat_list[out.widget.result]
if len(chosenlayer)==0:
    chosenlayer=wfs_ds.GetLayerByIndex(1).GetName()

layer = wfs_ds.GetLayerByName(chosenlayer)

print('chosen layer:',chosenlayer)

if not layer:
    sys.exit('ERROR: can not find layer in service')
else:
    pass

chosen layer: HS-c8427d7a30f0440b9773d17b7369fb69:watersheds


In [13]:
layer.GetFeatureCount()

33

In [36]:
%%time
count = 0
tdata = []

geometries=[]

print ('feature count: ', layer.GetFeatureCount())

if (layer.GetFeatureCount()>0):
#    feat=layer.GetFeature(1)
#    print(feat)

    for feat in layer:
        thisgeom = feat.geometry().GetGeometryName()

        if (thisgeom not in geometries):
            geometries.append(thisgeom)
        #ExportToJson doesn't work with some geometries (e.g .MULTISURFACE(??), 
        # in that case, replace with bounding box     
        try:
            temp=feat.ExportToJson()
        except:
            msenvelope=the_envelope( feat.geometry())
            #print('BoundingBox: %s' % msenvelope)
            thenote = 'Replace '+ thisgeom + ' with bounding box'
            if (thenote not in geometries):
                geometries.append(thenote)
            feat.SetGeometry(msenvelope)
        tdata.append(json.loads(feat.ExportToJson()))
        #feat = layer.GetNextFeature()
        count = count + 1
        #avoid getting hung on very long responses, maxsfeatures set in imports cell at top of notebook
        if count > maxfeatures:
            break
feat = None
print (geometries)

#print (tdata)
dataframe=json_normalize(tdata)
print('Dataframe size:',dataframe.size)

#print(dataframe)
print('done timed cell, %s features in result set' % count)

feature count:  33
['MULTISURFACE', 'Replace MULTISURFACE with bounding box']
Dataframe size: 231
       type  id geometry.type  \
0   Feature   1       Polygon   
1   Feature  10       Polygon   
2   Feature  11       Polygon   
3   Feature  12       Polygon   
4   Feature  13       Polygon   
5   Feature  14       Polygon   
6   Feature  15       Polygon   
7   Feature  16       Polygon   
8   Feature  17       Polygon   
9   Feature  18       Polygon   
10  Feature  19       Polygon   
11  Feature   2       Polygon   
12  Feature  20       Polygon   
13  Feature  21       Polygon   
14  Feature  22       Polygon   
15  Feature  23       Polygon   
16  Feature  24       Polygon   
17  Feature  25       Polygon   
18  Feature  26       Polygon   
19  Feature  27       Polygon   
20  Feature  28       Polygon   
21  Feature  29       Polygon   
22  Feature   3       Polygon   
23  Feature  30       Polygon   
24  Feature  31       Polygon   
25  Feature  32       Polygon   
26  Feature

Now get a list of the attributes of the feature type

Do some data summarization with pandas

#easy pandas way to get the field types
#dataframe.dtypes

In [37]:
lyrDefn = layer.GetLayerDefn()

print(dataframe.dtypes)

for col in dataframe.columns:
    if (col.find('.')>-1):
        field=str(col.split('.')[1])
        print('col: %s, field: %s' % (col, field))
        fielddef = lyrDefn.GetFieldDefn(lyrDefn.GetFieldIndex(field))
        if (fielddef is not None):
            fieldTypeCode = fielddef.GetType()
            fieldType = fielddef.GetFieldTypeName(fieldTypeCode)
            fieldWidth = fielddef.GetWidth()
            GetPrecision = fielddef.GetPrecision()       
            print (field + " - " + fieldType+ " " + str(fieldWidth) + " " + str(GetPrecision))
        else:
            print(field + ' has no ogr definition')

            
def findfieldtype(pcol):
    if (pcol.find('.')>-1):
        field=str(pcol.split('.')[1])
    else:
        field=str(pcol)
    fielddef = lyrDefn.GetFieldDefn(lyrDefn.GetFieldIndex(field))
    if (fielddef is not None):
        thetype= fielddef.GetFieldTypeName(fielddef.GetType())
    else:
        thetype='xxxx'
    return thetype

type                    object
id                       int64
geometry.type           object
geometry.coordinates    object
properties.gml_id       object
properties.id           object
properties.cat_id        int64
dtype: object
col: geometry.type, field: type
type has no ogr definition
col: geometry.coordinates, field: coordinates
coordinates has no ogr definition
col: properties.gml_id, field: gml_id
gml_id - String 0 0
col: properties.id, field: id
id - String 0 0
col: properties.cat_id, field: cat_id
cat_id - Integer64 0 0


In [38]:
print('%s features analyzed.' % len(dataframe.index))

print('Geometres found: %s' % geometries)

for col in dataframe.columns:
    #print(col)
    if (col.find('.')>-1):
        colname=str(col.split('.')[1])
    else:
        colname=str(col)
        
    if (dataframe[col].count() > 0) and  (dataframe[col].dtype=='float64'):
        mean=dataframe[col].mean()
        max=dataframe[col].max()
        min=dataframe[col].min()
        print('field:%s, count:%s, mean:%s; max:%s; min:%s \n' 
              % (colname, dataframe[col].count(), "{:.2f}".format(mean), 
                 "{:.2f}".format(max), "{:.2f}".format(min)))
    
    if (dataframe[col].count() > 0) and  (findfieldtype(col).find('Integer')>-1):
        max=dataframe[col].max()
        min=dataframe[col].min()
        print('field:%s, count:%s: max:%s; min:%s \n' 
              % (colname, dataframe[col].count(),  
                 "{:.2f}".format(max), "{:.2f}".format(min)))
    
    if (dataframe[col].count() > 0) and  (findfieldtype(col)=='String'):
        # and  (type(dataframe[col][1])=='str')
        #for item in dataframe[col].unique():
        #    print (item)
        #print (type(dataframe[col][1]))
        if len(dataframe.index)==len(dataframe[col].unique()):
            print(colname + ' is a text field, all values are unique')

        elif (len(dataframe[col].unique()) > 50):
            print(colname + ' appears to be a free text field, there are ' + str(len(dataframe[col].unique())) + ' unique values.' )

        else:
            print (colname +' has '+ str(len(dataframe[col].unique())) + ' unique values.') 
            print(dataframe[col].value_counts())
            
        print('\n')
 
emptyfields = []
print('\nThe following fields contain no data:')
for col in dataframe.columns:            
    if (dataframe[col].count() == 0):
        emptyfields.append(str(col.split('.')[1]))       
print(emptyfields)
        

33 features analyzed.
Geometres found: ['MULTISURFACE', 'Replace MULTISURFACE with bounding box']
id is a text field, all values are unique


gml_id is a text field, all values are unique


id is a text field, all values are unique


field:cat_id, count:33: max:20000043.00; min:20000011.00 


The following fields contain no data:
[]


In [39]:
 dataframe.describe()


,id,properties.cat_id
count,33.00000,3.300000e+01
mean,17.00000,2.000003e+07
std,9.66954,9.669540e+00
min,1.00000,2.000001e+07
25%,9.00000,2.000002e+07
50%,17.00000,2.000003e+07
75%,25.00000,2.000004e+07
max,33.00000,2.000004e+07
